# Optimizer 2022 R11 Solutions

### imports

In [11]:
%matplotlib inline
import math
import matplotlib.pyplot as plt
import seaborn as sns
import random
import plotly.express as px
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sympy import symbols, solve

### useful functions

In [12]:
def factorial(n):
    if n == 1:
        return 1
    return factorial(n-1)*n


def combinations(n, k):
    return factorial(n)/(factorial(n-k)*factorial(k))


class Point:
    def __init__(self, label, coords, deg=0, adj_list=None, error=0):
        if adj_list is None:
            adj_list = list()
        self.label = label
        self.coords = coords
        self.deg = deg
        self.adj_list = adj_list
        self.error = error
        
    def __repr__(self):
        return "point: %s\ndeg: %s" % (str(self.label), str(self.deg))


def read_input(input_dir):
    point_list = []
    with open(input_dir, 'r') as f:
        lines = f.readlines()
        d, n, m, k, rho = lines[0].strip().split()
        k_i = lines[1].strip().split()
        for line in lines[2:]:
            new = np.float32(line.strip().split())
            point_list.append(list(new))
    return [int(d), int(n) , int(m), int(k), int(rho), [int(e) for e in k_i]], point_list


def show_input_params(input_params):
    print("d: %i" % input_params[0])
    print("n: %i" % input_params[1])
    print("m: %i" % input_params[2])
    print("k: %i" % input_params[3])
    print("k_i: ", end="")
    print(" ".join(str(e) for e in input_params[5]))
    print("rho: %i" % input_params[4])
    return
    

def gng(points, max_iter, epsilon_b, epsilon_n, a_max, lambda_, alpha, D):
    w = []
    w_ab = random.sample(points, 2)
    for i in w_ab:
        w.append(i)
    it = 0
    while it < max_iter:
        x = random.sample(points, 1)
        x = x[0]
        min_dist = math.inf
        min_dist2 = math.inf
        w_s1_idx = -1
        w_s2_idx = -1
        w_s1 = 'a'
        w_s2 = 'a'
        for i in range(len(w)):
            d = math.dist(w[i].coords, x.coords)
            if d < min_dist:
                min_dist2 = min_dist
                w_s2_idx = w_s1_idx
                w_s2 = w_s1
                min_dist = d
                w_s1_idx = i
                w_s1 = w[i]
            elif d < min_dist2:
                min_dist2 = d
                w_s2_idx = i
                w_s2 = w[i]
        w[w_s1_idx].error += math.dist(w_s1.coords, x.coords)**2
        w[w_s1_idx].coords = tuple(np.asarray(w_s1.coords) + epsilon_b*(np.asarray(x.coords)-np.asarray(w_s1.coords)))
        for i in w[w_s1_idx].adj_list:
            w[i[0]].coords = tuple(np.asarray(w[i[0]].coords)+epsilon_n*(np.asarray(x.coords)-np.asarray(w[i[0]].coords)))
        if w_s2_idx not in w[w_s1_idx].adj_list:
            w[w_s1_idx].adj_list.append([w_s2_idx, 0])
            w[w_s2_idx].adj_list.append([w_s1_idx, 0])
            w[w_s1_idx].deg += 1
            w[w_s2_idx].deg += 1
        for p in w:
            i = 0
            while i < len(p.adj_list):
                if p.adj_list[i][1] > a_max:
                    del p.adj_list[i]
                i += 1
        i = 0
        while i < len(w):
            if w[i].deg == 0:
                del w[i]
            i += 1
        if it % lambda_ == 0:
            w_q_idx = -1
            max_err = 0
            for i in range(len(w)):
                if w[i].error < max_err:
                    max_err = w[i].error
                    w_q_idx = i
            w_r_idx = -1
            max_err = 0
            for p in w[w_q_idx].adj_list:
                if w[p[0]].error > max_err:
                    max_err = w[p[0]].error
                    w_r_idx = p[0]
            w_s_coords = tuple((np.asarray(w[w_q_idx].coords)+np.asarray(w[w_r_idx].coords))/2)
            w.append(Point(label=len(w)+1, coords=w_s_coords))
            w[-1].adj_list.append([w_q_idx, 0])
            w[-1].adj_list.append([w_r_idx, 0])
            w[1].deg += 2
            w[w_r_idx].adj_list.append([len(w)-1, 0])
            w[w_q_idx].adj_list.append([len(w)-1, 0])
            w[w_r_idx].deg += 1
            w[w_q_idx].deg += 1
            i = 0
            while i < len(w[w_q_idx].adj_list):
                if w[w_q_idx].adj_list[i][0] == w_r_idx:
                    del w[w_q_idx].adj_list[i]
                    break
                i += 1
            i = 0
            while i < len(w[w_r_idx].adj_list):
                if w[w_r_idx].adj_list[i][0] == w_q_idx:
                    del w[w_q_idx].adj_list[i]
                    break
                i += 1
            w[w_q_idx].error *= alpha
            w[w_r_idx].error *= alpha
            w[-1].error = w[w_q_idx].error

            for p in w:
                p.error *= D

#             if termination:
#                 break
        it += 1
    return w


def vector_quantization(method, points, n_output):
    if method.lower() == 'kmeans' or method.lower() == 'k-means':
        k = KMeans(n_clusters=n_output)
        k.fit(points)
        centers = k.cluster_centers_
        return centers


def dimentionality_reduction(points, old_dim, new_dim):
    pca = PCA(n_components=new_dim)
    new_points = pca.fit_transform(X=points)
    return new_points


def kmeans_clustering(points, k):
    k = KMeans(n_clusters=k)
    k.fit(points)
    centers = k.cluster_centers_
    labels = k.labels_
    return centers, labels


def attach_label_to_point(points, labels):
    df = create_df(points)
    df['label'] = labels
    return df.values


def quantized_to_origianl(points, clustered_quantized_points, centers):
    labels = points_to_centers_map(points, centers)
    final_labels = []
    for i in range(len(labels)):
        final_labels.append(clustered_quantized_points[labels[i], clustered_quantized_points.shape[1]-1])
    clustered = np.hstack(points, final_labels)
    return clustered
    
    
def points_to_centers_map(points, centers):
    labels = []
    for i in points:
        mn = math.inf
        l = -1
        for j in range(len(centers)):
            d = math.dist(i, centers[j])
            if d < mn:
                mn = d
                l = j
        labels.append(l)
    return labels


def lower_dimension(cluster:np.ndarray, dim, error):
    points = []
    for i in range(dim):
        points.append(cluster[i])
    A = np.asarray(points)
    b = np.ones((dim,))
    coefs = np.linalg.inv(A).dot(b)
    rnd = random.choice(cluster[dim:])
    if abs(coefs.dot(rnd) - 1) < error:
        return True, coefs/np.linalg.norm(coefs), rnd.dot(coefs/np.linalg.norm(coefs))
    return False, coefs/np.linalg.norm(coefs), rnd.dot(coefs/np.linalg.norm(coefs))


def lower_dimension_2(cluster:np.ndarray, dim, error):
    x = symbols('x')
    p1 = cluster[dim+10]
    p2 = cluster[dim+11]
    vec1 = p1 - p2
    vec2 = np.ones((dim-1,))
    vec2 = np.append(vec2, x)
    f = vec1.dot(vec2)
    s = solve(f)
    nor = np.ones((dim-1,))
    nor = np.append(nor, np.asarray(s))
    D = nor.dot(p1)
    rnd = random.choice(cluster[dim:])
    if abs(nor.dot(rnd) - D) < error:
        return True, nor, D
    return False, nor, D
    

def find_center_radius(cluster):
    diameter = 0
    for i in range(cluster.shape[0]):
        for j in range(i+1, cluster.shape[0]):
            p1 = cluster[i]
            p2 = cluster[j]
            d = math.dist(p1, p2)
            if d > diameter:
                diameter = d
                center = (p1+p2)/2
    return center, diameter/2


def index_list(df, k):
    total = [[] for i in range(k)]
    for i in range(len(df)):
        total[df['label'].iloc[i]].append(i+1)
    return total


def write_output(output_path, clustered_points):
    with open(output_path, 'w') as f:
        f.write('%s %s\n' % (str(n), str(k)))
        
        

def create_df(point_list):
    return pd.DataFrame(point_list)

### Reading input

In [13]:
input_params, point_list = read_input("./R11.txt")
org_df = create_df(point_list)
show_input_params(input_params)

d: 3
n: 300
m: 3
k: 3
k_i: 1 1 1
rho: 0


### Plots

In [14]:
fig = px.scatter_3d(dimensionality_reduced_points, x=0, y=1, z=2)
fig.show()

### Clustering procedure

#### Kmeans method

In [15]:
n_clusters = 3
centers, labels = kmeans_clustering(point_list, n_clusters)
org_df['label'] = labels

### Find center and radius of spheres

In [16]:
center0, radius0 = find_center_radius(p_list0)
center1, radius1 = find_center_radius(p_list1)
center2, radius2 = find_center_radius(p_list2)

### Export output

In [17]:
one = []
two = []
three = []
for i in range(len(org_df)):
    if org_df['label'].iloc[i] == 0:
        one.append(i)
    elif org_df['label'].iloc[i] == 1:
        two.append(i)
    else:
        three.append(i)

NameError: name 'df' is not defined

In [306]:
with open('output_R11.txt', 'w') as of:
    of.write("%d 3\n" % len(df))
    of.write("3 1 Sphere\n\n")
    of.write(" ".join(str(c) for c in list(center0)))
    of.write(" %f\n" % radius0)
    of.write("%d " % len(one))
    of.write(" ".join(str(c+1) for c in one))
    of.write("\n")
    of.write("3 1 Sphere\n\n")
    of.write(" ".join(str(c) for c in list(center1)))
    of.write(" %f\n" % radius1)
    of.write("%d " % len(two))
    of.write(" ".join(str(c+1) for c in two))
    of.write("\n")
    of.write("3 1 Sphere\n\n")
    of.write(" ".join(str(c) for c in list(center2)))
    of.write(" %f\n" % redius2)
    of.write("%d " % len(three))
    of.write(" ".join(str(c+1) for c in three))
    of.write("\n")
    of.write("0\n")